## Squats Counter Using Mediapipe

Features:
1. Initiation Action (Hands-up Position) 
2. Squat Posture (Knee Angle Calculator)
3. Voice Counter (Voice Feedback)

Importing Dependences

In [1]:
import cv2, mediapipe as mp , numpy as np , time
import pyttsx3
import threading


In [2]:
draw_utils = mp.solutions.drawing_utils
pose_model = mp.solutions.pose

Connections and Landmarks in MediaPipe

<img src = "https://google.github.io/mediapipe/images/mobile/pose_tracking_full_body_landmarks.png"></img>

In [3]:
def initiate_stance(left_hand_angle,right_hand_angle,flag = False):
    if flag == True:
        return True
    else:
        if left_hand_angle < 45 and right_hand_angle < 45:
            flag = True
            return True
    return False

In [4]:
def angle_between(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

In [5]:
def voice_counter(engine,x):
    engine.say(str(counter))
    engine.runAndWait()
    return 

# voice_counter(1)

In [6]:
def check_posture(knee_angle):
    connec = frozenset({(pose_model.PoseLandmark.LEFT_HIP, pose_model.PoseLandmark.LEFT_KNEE),
           (pose_model.PoseLandmark.RIGHT_HIP, pose_model.PoseLandmark.RIGHT_KNEE)})
    if min(knee_angle) < 85:
        draw_utils.draw_landmarks(img,results.pose_landmarks, connec,
                                draw_utils.DrawingSpec(color=(255,255,0), thickness=0, circle_radius=0), 
                                draw_utils.DrawingSpec(color=(0,0,250), thickness=5, circle_radius=3) 
                                )

In [ ]:
start = False
prev_time = 0
counter = 0 
stage = None

with pose_model.Pose(min_detection_confidence = 0.5,min_tracking_confidence = 0.5) as pose:
#     cap  = cv2.VideoCapture("Squats.mp4")
    cap  = cv2.VideoCapture(0)
    
    engine = pyttsx3.init()
    engine.setProperty("rate", 115)
    
    vw = int(cap.get(3))
    vh = int(cap.get(4))
    
#     To save results 
#     to_save = cv2.VideoWriter('Squats_result.avi', 
#                          cv2.VideoWriter_fourcc(*'MJPG'),
#                          10, (vw, vh))
    
    while cap.isOpened():
        ret, frame = cap.read()
        # Recolor Image
        img = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        
        img.flags.writeable = False
        
        # Running our model
        results = pose.process(img)
        
        # Recolor Image
        img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        img.flags.writeable = True
        
        # Render Detections
        
        draw_utils.draw_landmarks(img, results.pose_landmarks, pose_model.POSE_CONNECTIONS,
                                draw_utils.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=4), 
                                draw_utils.DrawingSpec(color=(0,150,0), thickness=3, circle_radius=3) 
                                )
        # Landmarks Extraction
        try:
            landmarks = results.pose_landmarks.landmark

        # Get coordinates
            
            # For Initiatin stance
            
            right_wrist    = [landmarks[pose_model.PoseLandmark.RIGHT_WRIST.value].x,landmarks[pose_model.PoseLandmark.RIGHT_WRIST.value].y]
            right_elbow    = [landmarks[pose_model.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[pose_model.PoseLandmark.RIGHT_ELBOW.value].y]
            right_shoulder = [landmarks[pose_model.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[pose_model.PoseLandmark.RIGHT_SHOULDER.value].y]

            left_wrist    = [landmarks[pose_model.PoseLandmark.LEFT_WRIST.value].x,landmarks[pose_model.PoseLandmark.LEFT_WRIST.value].y]
            left_elbow    = [landmarks[pose_model.PoseLandmark.LEFT_ELBOW.value].x,landmarks[pose_model.PoseLandmark.LEFT_ELBOW.value].y]
            left_shoulder = [landmarks[pose_model.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[pose_model.PoseLandmark.LEFT_SHOULDER.value].y]

            # For Squat Counter
            
            left_hip   = [landmarks[pose_model.PoseLandmark.LEFT_HIP.value].x,landmarks[pose_model.PoseLandmark.LEFT_HIP.value].y]
            left_knee  = [landmarks[pose_model.PoseLandmark.LEFT_KNEE.value].x,landmarks[pose_model.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[pose_model.PoseLandmark.LEFT_ANKLE.value].x,landmarks[pose_model.PoseLandmark.LEFT_ANKLE.value].y]

            right_hip   = [landmarks[pose_model.PoseLandmark.RIGHT_HIP.value].x,landmarks[pose_model.PoseLandmark.RIGHT_HIP.value].y]
            right_knee  = [landmarks[pose_model.PoseLandmark.RIGHT_KNEE.value].x,landmarks[pose_model.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[pose_model.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[pose_model.PoseLandmark.RIGHT_ANKLE.value].y]

            
            # Calculate angle
            
            angle_left      = angle_between(left_hip, left_knee, left_ankle)
            angle_right     = angle_between(right_hip, right_knee, right_ankle)
            angle_righthand = angle_between(right_wrist,right_elbow,right_shoulder)
            angle_lefthand  = angle_between(left_wrist,left_elbow,left_shoulder)
#           print(angle_left,angle_right)  --> checking for angle
    
            # Visualize knee_angle
            cv2.putText(img, str(f"{min(angle_left,angle_right):0.2f}"), 
                           tuple(np.multiply(left_knee, [vw*0.85,vh]).astype('int32')), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100, 150, 160), 2, cv2.LINE_AA
                                )
            # Initiate Squats
            
            if initiate_stance(angle_lefthand,angle_righthand):
                check_posture([angle_left,angle_right])
                
            # Curl counter logic
                if angle_left > 160 and angle_right> 160:
                    stage = "DOWN"
                elif angle_left < 97 and angle_right <80 and stage =='DOWN':#due to angle of the video the left side angle is higher than right
                    stage="UP"
                    counter +=1
                    thread  = threading.Thread(target = voice_counter, args = [engine,counter]) # Voice output
                    thread.start()
        #         print(counter)
        except:
            pass
         
    # Display Data
        
        # Fps
        curr_time = time.time()
        fps = 1//(curr_time-prev_time)
        prev_time = curr_time
        
        cv2.putText(img, "FPS: "+str(fps), (550,10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,0,255), 1, cv2.LINE_AA)
        
        # Counter Background
        cv2.rectangle(img, (0,0), (225,73), (245,117,16), -1)
        
        # Squat Counter
        cv2.putText(img, 'REPS', (10,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA) # Title
        cv2.putText(img, str(counter), 
                    (13,50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA) # Count
        
        # Guiding Stage
        cv2.putText(img, 'GO', (70,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA) # Title
        cv2.putText(img, stage, 
                    (65,50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)  # Direction to move  
        
        
        
        # Display img
#         to_save.write(img)
        
        cv2.imshow("Mediapipe Feed",img)
    # Exit Loop when pressed e
        if cv2.waitKey(2) & 0xFF == ord('e'):
            break
cap.release()
# to_save.release()
cv2.destroyAllWindows()